link for data https://drive.google.com/drive/folders/1T0f90mbUv7VuI9Zu4eOLVIRCxzJ2iJG1?usp=sharing

In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
from dash import Dash, html, dcc, callback, Output, Input

In [ ]:

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json


In [ ]:
movies_genres = pd.read_csv('genres.csv')
movies_countries = pd.read_csv('countries.csv')
ratings = pd.read_csv('Ratings.csv')

In [ ]:
country_counts = movies_countries.groupby('country').size().reset_index(name='movie_count')


In [ ]:
merged_genres = pd.merge(movies_genres, movies_countries, on='id')
merged_genres = pd.merge(merged_genres, ratings, on='id')

merged_genres['country'].replace(
    to_replace=['Russian Federation', 'USSR'],
    value='Russia',
    inplace=True
)

merged_genres['genre'].replace(
    to_replace=['Horror', 'Thriller'],
    value='Horror & Thriller',
    inplace=True
)

merged_genres['genre'].replace(
    to_replace=['Mystery', 'Crime'],
    value='Mystery & Crime',
    inplace=True
)

merged_genres['genre'].replace(
    to_replace=['Fantasy', 'Science Fiction'],
    value='Fantasy & Science Fiction',
    inplace=True
)

merged_genres['genre'].replace(
    to_replace=['History', 'War', 'Western'],
    value='History, War, & Western',
    inplace=True
)

grouped_genres = merged_genres.groupby(['country', 'genre']).size().reset_index(name='count')
idx = grouped_genres.groupby(['country'])['count'].transform(max) == grouped_genres['count']
highest_count_genres = grouped_genres[idx]
total_movies_per_country = grouped_genres.groupby('country')['count'].sum()
highest_count_genres['percentage'] = highest_count_genres.apply(
    lambda row: (row['count'] / total_movies_per_country[row['country']]) * 100,
    axis=1
)
highest_count_genres['percentage'] = highest_count_genres['percentage'].round(2)

In [ ]:
idx.head()

,count
0,False
1,False
2,False
3,True
4,False


In [ ]:
genres_ratings = pd.pivot_table(merged_genres,values='rating', index=['country','genre'], aggfunc='mean').reset_index()
genres_ratings.head()

,country,genre,rating
0,Afghanistan,Action,3.230000
1,Afghanistan,Adventure,3.230000
2,Afghanistan,Comedy,3.880000
3,Afghanistan,Documentary,3.505833
4,Afghanistan,Drama,3.475455


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
countries_geojson = ['https://github.com/johan/world.geo.json/blob/master/countries.geo.json']

Genres_app = Dash()



# Requires Dash 2.17.0 or later
Genres_app.layout = html.Div([
    html.H1(children='Genres per Country', style={'textAlign':'center'}),

    html.Div([

    dcc.Graph(id="choropleth-country"),
    dcc.Graph(id="choropleth-graph")
  ], style={'width': '49%', 'display': 'inline-block'}
           ),


  html.Div([
   dcc.Dropdown(
                grouped_genres['country'].unique(),
                'USA',
                id='crossfilter-xaxis-column',
            ),

    html.Hr(),
    html.Hr(),



    html.Div(id='display-selected-values'),

    dcc.Graph(id="bar-graph"),
    dcc.Graph(id="bar-ratings"),
  ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}
           )
])

@Genres_app.callback( # Columns 2m_temp_prod, or....
    Output('choropleth-country', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
  fig = px.choropleth(
    country_counts,
    locations='country',
    locationmode='country names',
    color='movie_count',
    color_continuous_scale='reds',
    title='Distribution of Movies by Country'
)


  fig.update_layout(
    coloraxis_colorbar=dict(title='Number of Movies')
)


  fig.update_layout(
    width=700,
    height=500,
)
  return fig

@Genres_app.callback( # Columns 2m_temp_prod, or....
    Output('choropleth-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = highest_count_genres[highest_count_genres.country==value]
    return px.choropleth(
    highest_count_genres,
    geojson= countries_geojson,
    locations='country',
    locationmode='country names',
    color='genre',
    hover_data=['count', 'percentage'],
    title='Genre with Highest Count by Country',
    color_discrete_sequence=px.colors.qualitative.Set2
)

@Genres_app.callback( # Columns 2m_temp_prod, or....
    Output('bar-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = grouped_genres[grouped_genres.country==value]
    return px.bar(dff, x="country", y="count", barmode = 'group', color = 'genre', text="genre", title="Genre Count per Country")

@Genres_app.callback( # Columns 2m_temp_prod, or....
    Output('bar-ratings', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dfg = genres_ratings[genres_ratings.country==value]
    return px.bar(dfg, x="country", y="rating", barmode = 'group', color="genre", text="genre", title="Average Genre Ratings per Country")

if __name__ == '__main__':
    Genres_app.run(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
movies_releases = pd.read_csv('releases.csv')

merged_releases = pd.merge(movies_releases, movies_countries, on='id')
merged_releases = pd.merge(merged_releases, ratings, on='id')

merged_genres['country'].replace(
    to_replace=['Russian Federation', 'USSR'],
    value='Russia',
    inplace=True
)



grouped_releases = merged_releases.groupby(['country', 'type']).size().reset_index(name='count')
idx = grouped_releases.groupby(['country'])['count'].transform(max) == grouped_releases['count']
highest_count_releases = grouped_releases[idx]
total_releases_per_country = grouped_releases.groupby('country')['count'].sum()
highest_count_releases['percentage'] = highest_count_releases.apply(
    lambda row: (row['count'] / total_releases_per_country[row['country']]) * 100,
    axis=1
)
highest_count_releases['percentage'] = highest_count_releases['percentage'].round(2)

In [ ]:
releases_ratings = pd.pivot_table(merged_releases,values='rating', index=['country','type'], aggfunc='mean').reset_index()
releases_ratings.head()

,country,type,rating
0,Afghanistan,Digital,3.384545
1,Afghanistan,Physical,3.520000
2,Afghanistan,Premiere,3.518108
3,Afghanistan,Theatrical,3.506136
4,Afghanistan,Theatrical limited,3.465000


In [ ]:
grouped_releases.head()

,country,type,count
0,Afghanistan,Digital,28
1,Afghanistan,Physical,3
2,Afghanistan,Premiere,81
3,Afghanistan,TV,5
4,Afghanistan,Theatrical,127


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
countries_geojson = ['https://github.com/johan/world.geo.json/blob/master/countries.geo.json']

app = Dash()



# Requires Dash 2.17.0 or later
app.layout = html.Div([
  html.H1(children='Release Types per Country', style={'textAlign':'center'}),

  html.Div([

    dcc.Graph(id="choropleth-graph")
  ], style={'width': '49%', 'display': 'inline-block'}
           ),



  html.Div([
   dcc.Dropdown(
                grouped_releases['country'].unique(),
                'USA',
                id='crossfilter-xaxis-column',
            ),

    html.Hr(),
    html.Hr(),



    html.Div(id='display-selected-values'),

    dcc.Graph(id="bar-graph"),
    dcc.Graph(id="bar-ratings"),
  ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}
           )
])


@app.callback( # Columns 2m_temp_prod, or....
    Output('choropleth-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = highest_count_releases[highest_count_releases.country==value]
    return px.choropleth(
    highest_count_releases,
    geojson= countries_geojson,
    locations='country',
    locationmode='country names',
    color='type',
    hover_data=['count', 'percentage'],
    title='Release Type with Highest Count by Country',

    color_discrete_sequence=px.colors.qualitative.Set2
)

@app.callback( # Columns 2m_temp_prod, or....
    Output('bar-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = grouped_releases[grouped_releases.country==value]
    return px.bar(dff, x="country", y="count", barmode = 'group', color = 'type', text="type", title="Release Type Count per Country")

@app.callback( # Columns 2m_temp_prod, or....
    Output('bar-ratings', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dfg = releases_ratings[releases_ratings.country==value]
    return px.bar(dfg, x="country", y="rating", barmode = 'group', color="type", text="type", title="Average Release Type Ratings per Country")

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
movies_themes = pd.read_csv('themes.csv')
movies_themes.head()

,id,theme
0,1000001,Humanity and the world around us
1,1000001,Crude humor and satire
2,1000001,Moving relationship stories
3,1000001,Emotional and captivating fantasy storytelling
4,1000001,Surreal and thought-provoking visions of life ...


In [ ]:
movies_themes = pd.read_csv('themes.csv')

merged_themes = pd.merge(movies_countries, movies_themes, on='id')
merged_themes = pd.merge(merged_themes, ratings, on='id')

merged_themes['country'].replace(
    to_replace=['Russian Federation', 'USSR'],
    value='Russia',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['War and historical adventure',
                'Nazis and World War II',
                'Military combat and heroic soldiers',
                'Bravery in War',
                'Engaging, intense crime and casino drama',
                'Violent crime and drugs',
                'Brutal, violent prison drama',
                'Crime, drugs and gangsters',
                'Military combat and heroic soldiers',
                'Fascinating, emotional stories and documentaries',
                'Gritty crime and ruthless gangsters',
                'Lavish dramas and sumptuous royalty',
                'Humanity and the world around us',
                'Intense violence and sexual transgression',
                'Political drama, patriotism, and war',
                'Politics, propaganda, and political documentaries',
                'Riveting political and presidential drama',
                'Politics and human rights',
                'Intense political and terrorist thrillers',
                'Racism and the powerful fight for justice',
                'Emotional LGBTQ relationships',
                'Faith and religion',
                'Faith and spiritual journeys',
                'Religious faith, sin, and forgiveness'
                ],
    value='Politics, Relegion, Society',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Emotional teen coming-of-age stories',
                'Underdogs and coming of age',
                'Teen friendship and coming-of-age',
                'Student coming-of-age challenges',
                'Inspiring sports underdog stories',
                'Underdog fighting and boxing stories',
                'Teen school antics and laughter'
                ],
    value='Coming of Age',
    inplace=True
)


merged_themes['theme'].replace(
    to_replace=['Passion and romance',
                'Adorable animals and heartwarming families',
                'Emotional and touching family dramas',
                'Erotic relationships and desire',
                'Relationship comedy',
                'Charming romances and delightful chemistry',
                'Captivating relationships and charming romance',
                'Challenging or sexual themes & twists',
                'Touching and sentimental family stories',
                'Enduring stories of family and marital drama',
                'Heartbreaking and moving family drama',
                'Quirky and endearing relationships',
                'Laugh-out-loud relationship entanglements',
                'Moving relationship stories'
                ],
    value='Love, Family, & Relationships',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Epic heroes',
                'Action comedy and silly heroics',
                'Fantasy adventure, heroism, and swordplay',
                'Historical battles and epic heroism',
                'Superheroes in action-packed battles with villains',
                'Explosive and action-packed heroes vs. villains',
                ],
    value='Good vs. Evil',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Sci-fi monster and dinosaur adventures',
                'Sci-fi horror, creatures, and aliens',
                'Action-packed space and alien sagas',
                'Thought-provoking sci-fi action and future technology',
                'Imaginative space odysseys and alien encounters',
                'Monsters, aliens, sci-fi and the apocalypse',
                'Dangerous technology and the apocalypse',
                'Fairy-tale fantasy and enchanted magic',
                "Kids' animated fun and adventure",
                'Emotional and captivating fantasy storytelling'
                ],
    value='Technology & Supernatural',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Legendary musicians and stardom',
                'Holiday joy and heartwarming Christmas',
                'Emotional life of renowned artists',
                'Dreamlike, quirky, and surreal storytelling',
                'Surreal and thought-provoking visions of life and death',
                'Powerful stories of heartbreak and suffering',
                'Tragic sadness and captivating beauty',
                'Captivating vision and Shakespearean drama',
                'Powerful poetic and passionate drama'
                ],
    value='Virtues, Values, & Tragedy',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Air pilot heroism and survival',
                'Disastrous voyages and heroic survival',
                "Humanity's odyssey: earth and beyond",
                'Explosive and action-packed heroes vs. villains',
                'Survival horror and zombie carnage',
                'Graphic violence and brutal revenge',
                'Epic adventure and breathtaking battles',
                'Epic history and literature',
                'Westerns',
                'Bollywood emotional dramas'
                ],
    value='History, Survival, & Human Nature',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Extreme gory horror and cannibalism',
                'Bloody vampire horror',
                'Chilling experiments and classic monster horror',
                'Gothic and eerie haunting horror',
                'Gory, gruesome, and slasher horror',
                'Creepy, chilling, and terrifying horror',
                'Terrifying, haunted, and supernatural horror',
                'Horror, the undead and monster classics',
                ],
    value='Gore, Horror, & Thriller',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=[
                'Suspenseful crime thrillers',
                'Intriguing and suspenseful murder mysteries',
                'Noir and dark crime dramas',
                'Thrillers and murder mysteries',
                'Twisted dark psychological thriller'
                ],
    value='Mystery & Suspense',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Crude humor and satire',
                'Amusing jokes and witty satire',
                'Gags, jokes, and slapstick humor',
                'Funny jokes and crude humor',
                'Western frontier dramas with a touch of humor',
                'Catchy songs and hilarious musical comedy',
                'Spooky, scary comedy',
                'Song and dance',
                'Dazzling vocal performances and musicals',
                'Dance rhythms and catchy tunes'
                ],
    value='Happiness and Humor',
    inplace=True
)

merged_themes['theme'].replace(
    to_replace=['Gripping, intense violent crime',
                'Violent action, guns, and crime',
                'Heists and thrilling action',
                'Adrenaline-fueled action and fast cars',
                'Intense combat and martial arts',
                'Wild west outlaws and gunfights',
                'Exciting spy thrillers with tense intrigue',
                'High speed and special ops'
                ],
    value='Action & Violence',
    inplace=True
)

grouped_themes = merged_themes.groupby(['country', 'theme']).size().reset_index(name='count')
idx = grouped_themes.groupby(['country'])['count'].transform(max) == grouped_themes['count']
highest_count_themes = grouped_themes[idx]
total_themes_per_country = grouped_themes.groupby('country')['count'].sum()
highest_count_themes['percentage'] = highest_count_themes.apply(
    lambda row: (row['count'] / total_themes_per_country[row['country']]) * 100,
    axis=1
)
highest_count_themes['percentage'] = highest_count_themes['percentage'].round(2)

In [ ]:
themes_ratings = pd.pivot_table(merged_themes,values='rating', index=['country','theme'], aggfunc='mean').reset_index()
themes_ratings.head()

,country,theme,rating
0,Afghanistan,Good vs. Evil,3.290000
1,Afghanistan,"History, Survival, & Human Nature",3.870000
2,Afghanistan,"Love, Family, & Relationships",3.680000
3,Afghanistan,"Politics, Relegion, Society",3.451667
4,Afghanistan,Technology & Supernatural,3.870000


In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
countries_geojson = ['https://github.com/johan/world.geo.json/blob/master/countries.geo.json']

app = Dash()



# Requires Dash 2.17.0 or later
app.layout = html.Div([
 html.H1(children='Grouped Themes Types per Country', style={'textAlign':'center'}),

    html.Div([
    dcc.Graph(id="choropleth-graph")
  ], style={'width': '49%', 'display': 'inline-block'}
           ),



  html.Div([
   dcc.Dropdown(
                grouped_themes['country'].unique(),
                'USA',
                id='crossfilter-xaxis-column',
            ),

    html.Hr(),
    html.Hr(),



    html.Div(id='display-selected-values'),

    dcc.Graph(id="bar-graph"),
    dcc.Graph(id="bar-ratings"),
  ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}
           )
])


@app.callback( # Columns 2m_temp_prod, or....
    Output('choropleth-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = highest_count_themes[highest_count_themes.country==value]
    return px.choropleth(
    highest_count_themes,
    geojson= countries_geojson,
    locations='country',
    locationmode='country names',
    color='theme',
    hover_data=['count', 'percentage'],
    title='Grouped Themes with Highest Count by Country',

    color_discrete_sequence=px.colors.qualitative.Set2
)

@app.callback( # Columns 2m_temp_prod, or....
    Output('bar-graph', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dff = grouped_themes[grouped_themes.country==value]
    return px.bar(dff, x="country", y="count", barmode = 'group', color = 'theme', text="theme", title="Theme Count per Country")

@app.callback( # Columns 2m_temp_prod, or....
    Output('bar-ratings', 'figure'),
    [Input('crossfilter-xaxis-column', 'value'),
     ]
)
def update_genre_graph(value):
    dfg = themes_ratings[themes_ratings.country==value]
    return px.bar(dfg, x="country", y="rating", barmode = 'group', color="theme", text="theme", title="Average Theme Ratings per Country")

if __name__ == '__main__':
    app.run(debug=True)

<IPython.core.display.Javascript object>